# HR & IT Company AI Agent with LangChain and Mistral

This notebook demonstrates a conversational HR/IT agent for an IT company using LangChain, Mistral LLM, and 8 advanced tools.

**Agent Capabilities:**
- Send onboarding and other emails
- Answer HR policy questions using RAG
- Employee lookup
- Leave request processing
- Contextual email generation
- IT equipment requests
- Performance review scheduling
- Full conversational memory

## 1. Setup & Dependencies
Install required libraries and load API keys.

In [ ]:
%pip install langchain-mistralai
%pip install python-dotenv
%pip install getpass
%pip install langchain
%pip install chromadb
%pip install langchain-community

In [ ]:
import os
from dotenv import load_dotenv
from getpass import getpass
load_dotenv()
MISTRAL_API_KEY = os.getenv('MISTRAL_API_KEY')
if not MISTRAL_API_KEY:
    MISTRAL_API_KEY = getpass('Enter your Mistral API key: ')
if MISTRAL_API_KEY:
    print('✅ Mistral API key loaded.')
else:
    raise ValueError('❌ Mistral API key not found. Please set it in your .env file or enter it when prompted.')

## 2. Configure Mistral LLM in LangChain

In [ ]:
from langchain_mistralai.chat_models import ChatMistralAI
MISTRAL_MODEL = 'mistral-medium'
llm = ChatMistralAI(api_key=MISTRAL_API_KEY, model=MISTRAL_MODEL)
print(f'✅ Mistral LLM configured with model: {MISTRAL_MODEL}')

## 3. HR & IT Company Tool Definitions and Mock Data
All mock data, tool functions, and tool definitions are grouped here for clarity.

In [ ]:
from langchain.tools import Tool
import re
from typing import List
from datetime import datetime, timedelta
import json
def send_welcome_email(info: str) -> str:
    required_fields = ['name', 'email', 'position', 'start_date']
    missing = [field for field in required_fields if not re.search(f'{field}\s*:', info, re.IGNORECASE)]
    if missing:
        return f'Missing info: {', '.join(missing)}. Please provide as: name: John Doe, email: john@company.com, position: Intern, start_date: 2024-08-01.'
    return f'Welcome email sent to {info}.'
send_welcome_email_tool = Tool(
    name='SendWelcomeEmailTool',
    func=send_welcome_email,
    description="Send a welcome email to a new employee or intern. Input should include: name, email, position, start_date."
)
# --- Tool 2: SendOtherEmailTool ---
def send_other_email(info: str) -> str:
    return f'Other onboarding email sent with info: {info}.'
send_other_email_tool = Tool(
    name='SendOtherEmailTool',
    func=send_other_email,
    description="Send other onboarding emails (e.g., IT setup, HR forms). Input should include recipient and purpose."
)
# --- Tool 3: HRPolicyQATool (RAG) ---
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import TextLoader, CSVLoader, JSONLoader, PyPDFLoader
def load_hr_docs(doc_paths: List[str]):
    docs = []
    for path in doc_paths:
        ext = os.path.splitext(path)[1].lower()
        if ext == '.pdf': loader = PyPDFLoader(path)
        elif ext == '.csv': loader = CSVLoader(path)
        elif ext == '.json': loader = JSONLoader(path)
        elif ext == '.txt': loader = TextLoader(path)
        else: continue
        docs.extend(loader.load())
    return docs
hr_docs_folders = ['hr_docs', 'docs']
doc_paths = []
for folder in hr_docs_folders:
    if os.path.exists(folder):
        doc_paths += [os.path.join(folder, f) for f in os.listdir(folder)]
hr_docs = load_hr_docs(doc_paths) if doc_paths else []
if not hr_docs:
    from langchain.schema import Document
    hr_docs = [Document(page_content='Employees are eligible for 20 days of paid leave per year.'), Document(page_content='Interns must submit timesheets weekly.'), Document(page_content='Remote work is allowed up to 3 days per week.')]
embeddings = HuggingFaceEmbeddings()
vectorstore = FAISS.from_documents(hr_docs, embedding=embeddings)
retriever = vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
def hr_policy_qa(question: str) -> str:
    return qa_chain.run(question)
hr_policy_qa_tool = Tool(
    name='HRPolicyQATool',
    func=hr_policy_qa,
    description="Answer questions about HR policies using RAG from HR docs (supports PDF, CSV, JSON, TXT)."
)
# --- Tool 4: EmployeeLookupTool ---
def employee_lookup(query: str) -> str:
    query_lower = query.lower()
    results = []
    for emp in employees:
        if (query_lower in emp['name'].lower() or
            query_lower in emp['id'].lower() or
            query_lower in emp['department'].lower() or
            query_lower in emp['role'].lower() or
            any(query_lower in skill.lower() for skill in emp['skills']) or
            query_lower in emp['manager'].lower()):
            results.append(emp)
    if not results:
        # Fuzzy/partial match
        for emp in employees:
            if any(word in emp['name'].lower() for word in query_lower.split()):
                results.append(emp)
    if not results:
        return 'No matching employee found.'
    return json.dumps(results, indent=2)
employee_lookup_tool = Tool(
    name='EmployeeLookupTool',
    func=employee_lookup,
    description="Searches the employee database by name, ID, department, role, skills, or manager. Returns contact info, department, manager, start date, and current projects. Handles partial and fuzzy matches."
)
# --- Tool 5: LeaveRequestTool ---
def leave_request(input_str: str) -> str:
    pattern = r'(vacation|sick|pto|personal)\s+for\s+([\w\s]+)\s+(\w+\s*\d{1,2}(-|to)?\s*\w*\d{0,2}|\d{4}-\d{2}-\d{2}(\s*to\s*\d{4}-\d{2}-\d{2})?)?(.*)'
    match = re.search(pattern, input_str, re.IGNORECASE)
    if not match:
        return 'Could not parse leave request. Please use format: "Vacation for Alice Johnson Dec 15-20 because family trip".'
    leave_type = match.group(1).capitalize()
    name = match.group(2).strip()
    date_info = match.group(3) or ''
    reason = match.group(6).strip() if match.group(6) else ''
    balance = leave_balances.get(name, None)
    if balance is None:
        return f'Employee {name} not found.'
    # Parse dates
    try:
        if '-' in date_info or 'to' in date_info:
            # Range
            dates = re.split(r'-|to', date_info)
            start = dates[0].strip()
            end = dates[1].strip() if len(dates) > 1 else start
            # Try to parse as YYYY-MM-DD
            try:
                start_date = datetime.strptime(start, '%Y-%m-%d')
                end_date = datetime.strptime(end, '%Y-%m-%d')
            except:
                # Try as Month Day
                start_date = datetime.strptime(start + f' {datetime.now().year}', '%b %d %Y')
                end_date = datetime.strptime(end + f' {datetime.now().year}', '%b %d %Y')
        else:
            # Single day
            try:
                start_date = datetime.strptime(date_info, '%Y-%m-%d')
            except:
                start_date = datetime.strptime(date_info + f' {datetime.now().year}', '%b %d %Y')
            end_date = start_date
    except Exception as e:
        return f'Could not parse dates: {e}'
    days = (end_date - start_date).days + 1
    if days > balance:
        return f'{name} has only {balance} days left. Requested: {days} days.'
    return f'{leave_type} leave for {name} from {start_date.date()} to {end_date.date()} ({days} days) approved. Reason: {reason}'
leave_request_tool = Tool(
    name='LeaveRequestTool',
    func=leave_request,
    description="Processes vacation/sick/personal leave requests. Parses natural language, validates dates, leave balance, and conflicts. Returns confirmation and warnings."
)
# --- Tool 6: TemplateEmailTool ---
def template_email(input_str: str) -> str:
    # Example: 'welcome_developer name: Alice Johnson role: Frontend Developer start_date: 2023-02-15'
    parts = input_str.split()
    template_key = parts[0]
    args = {}
    for part in parts[1:]:
        if ':' in part:
            k, v = part.split(':', 1)
            args[k.strip()] = v.strip()
    template = email_templates.get(template_key, None)
    if not template:
        return f'No template found for {template_key}'
    try:
        return template.format(**args)
    except Exception as e:
        return f'Error formatting template: {e}'
template_email_tool = Tool(
    name='TemplateEmailTool',
    func=template_email,
    description="Generates contextual emails using IT company templates. Accepts template type, recipient info, and dynamic variables. Personalizes by role and scenario."
)
# --- Tool 7: ITEquipmentRequestTool ---
def it_equipment_request(input_str: str) -> str:
    input_lower = input_str.lower()
    for item in equipment_inventory:
        if item in input_lower:
            if equipment_inventory[item] > 0:
                equipment_inventory[item] -= 1
                return f'{item.capitalize()} request approved. {equipment_inventory[item]} remaining in inventory.'
            else:
                return f'{item.capitalize()} is out of stock.'
    # Software access
    if 'figma' in input_lower:
        return 'Figma access granted.'
    if 'docker' in input_lower:
        return 'Docker setup completed.'
    if 'postgresql' in input_lower:
        return 'PostgreSQL access granted.'
    return 'No matching equipment or software found.'
it_equipment_request_tool = Tool(
    name='ITEquipmentRequestTool',
    func=it_equipment_request,
    description="Handles hardware/software requests and recommends equipment based on role and project needs. Tracks inventory and provides delivery/setup info."
)
# --- Tool 8: PerformanceReviewTool ---
def performance_review(input_str: str) -> str:
    # Example: 'Set up quarterly review for Alice Johnson on 2024-09-01'
    pattern = r'(review|performance|check|setup|schedule).*for\s+([\w\s]+)(.*)'
    match = re.search(pattern, input_str, re.IGNORECASE)
    if not match:
        return 'Could not parse review request. Please use format: "Set up quarterly review for Alice Johnson on 2024-09-01".'
    name = match.group(2).strip()
    rest = match.group(3).strip()
    date_match = re.search(r'(\d{4}-\d{2}-\d{2})', rest)
    date = date_match.group(1) if date_match else None
    emp = next((e for e in employees if e['name'].lower() == name.lower()), None)
    if not emp:
        return f'Employee {name} not found.'
    role = emp['role'].lower().split()[0]
    template = review_templates.get(role, None)
    if not template:
        return f'No review template for role {role}'
    if date:
        if name in review_schedules:
            review_schedules[name].append(date)
        else:
            review_schedules[name] = [date]
        return template.format(name=name, role=emp['role'], date=date)
    else:
        # Show next scheduled review
        next_review = review_schedules.get(name, ['No review scheduled'])[0]
        return f'Next review for {name} ({emp['role']}): {next_review}'
performance_review_tool = Tool(
    name='PerformanceReviewTool',
    func=performance_review,
    description="Schedules and manages performance reviews. Generates role-specific templates, tracks schedules, and provides career development suggestions."
)
# --- Tools List ---
tools = [send_welcome_email_tool, send_other_email_tool, hr_policy_qa_tool, employee_lookup_tool, leave_request_tool, template_email_tool, it_equipment_request_tool, performance_review_tool]
print('✅ All HR & IT company tools loaded.')

<>:51: SyntaxWarning: invalid escape sequence '\s'
<>:51: SyntaxWarning: invalid escape sequence '\s'
C:\Users\diyak\AppData\Local\Temp\ipykernel_23640\3027019105.py:51: SyntaxWarning: invalid escape sequence '\s'
  missing = [field for field in required_fields if not re.search(f'{field}\s*:', info, re.IGNORECASE)]


TypeError: JSONLoader.__init__() missing 1 required positional argument: 'jq_schema'

## 4. Agent Initialization and Memory

In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)
print('✅ HR & IT agent initialized with all tools and memory!')

## 5. Example Usage and Testing
Demonstrates queries for all tools and agent.

In [ ]:
# --- Example Usage and Testing ---
print('--- EmployeeLookupTool Example ---')
print(employee_lookup_tool.func('Who is the Python developer in Engineering?'))
print(employee_lookup_tool.func('Find Sarah from design'))
print(employee_lookup_tool.func('Show me all QA engineers'))
print('--- LeaveRequestTool Example ---')
print(leave_request_tool.func('Vacation for Alice Johnson Dec 15-20 because family trip'))
print(leave_request_tool.func('Sick leave for Bob Smith tomorrow'))
print(leave_request_tool.func('PTO for Emily Chen 2024-12-24 to 2024-12-31'))
print('--- TemplateEmailTool Example ---')
print(template_email_tool.func('welcome_developer name: Alice Johnson role: Frontend Developer start_date: 2023-02-15'))
print(template_email_tool.func('it_setup name: Bob Smith role: Backend Developer setup: Docker, PostgreSQL'))
print(template_email_tool.func('security_training name: Sarah Kim date: 2024-08-01'))
print('--- ITEquipmentRequestTool Example ---')
print(it_equipment_request_tool.func('I need a laptop for machine learning'))
print(it_equipment_request_tool.func('Figma access for design work'))
print(it_equipment_request_tool.func('Monitor for Olivia Wang'))
print('--- PerformanceReviewTool Example ---')
print(performance_review_tool.func('Set up quarterly review for Alice Johnson on 2024-09-01'))
print(performance_review_tool.func('Performance check for Sarah Kim'))
# Example queries to test RAG with new docs
print('Testing RAG with new documents in docs folder:')
test_queries = [
    "What is the company's remote work policy?",
    "How many paid leave days do employees get?",
    "What is the process for submitting timesheets?",
    "What is the dress code for interns?",
    "Who should I contact for IT support during onboarding?",
    "Are there any special HR policies for new hires in 2025?"
 ]
for i, q in enumerate(test_queries, 1):
    print(f'Query {i}: {q}')
    print('Answer:', agent.run(q))
    print('-' * 60)